### `Linear` code

In [1]:
import numpy as np
import torch # needed for tests
from tqdm import tqdm # needed for tests

In [29]:
class Linear:
    ''' Applies a linear transformation given the input '''
    
    ''' * The class implementation will be along the lines of torch.nn.Linear in order to 
          enable comparison of this NumPy only implementation and seamless testing
        * Can expect extensive refactoring of the existing code in the days to come
        * As part of refactoring, some code will be de-modularized
        * Old code will be retained at the end of the notebook for reference
    '''
    '''
        TODO:
        * Replace `torch.round()` with `np.allclose()` for tests
        * Optimizing code
    '''
    
    def __init__(
        self,
        in_features,
        out_features,
        bias = True,
        device = None,
        dtype = None,
        verbose = False
        ):
        super(Linear, self).__init__()
        
        ''' mandatory parameters '''
        self.in_features = in_features
        self.out_features = out_features
        
        ''' optional parameters '''
        # None
        
        ''' optional parameters (dummy, yet to be implemented)'''
        self.bias = bias
        self.device = device
        self.dtype = dtype
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.verboseprint('*** parameters ***')
        self.verboseprint('in_features: {}, out_features: {}, bias: {}'.format(self.in_features, self.out_features, self.bias))
        self.verboseprint('device: {}, dtype: {}'.format(self.device, self.dtype))
        self.verboseprint('\n')
    
    def forward(self, _input, weights = None, bias_weights = None):
        ''' forward pass to perform a linear transformation '''
        
        ''' error checking '''
        if not (isinstance(_input, int) or isinstance(_input, float) or isinstance(_input, np.ndarray)):
            raise Exception('invalid input: input should either be an int, a float, or a NumPy ndarray')
        
        ''' use the provided weights or create random weights based on the input parameters '''
        if weights is not None:
            self.verboseprint('*** weights ***')
            self.verboseprint('weights shape: {}'.format(weights.shape))
        else:
            weights = np.random.rand(self.out_features, self.in_features)
        self.verboseprint(weights)
        self.verboseprint('\n')                      
                              
        if self.bias:
            if bias_weights is not None:
                self.verboseprint('*** bias ***')
                self.verboseprint('bias shape: {}'.format(bias_weights.shape))
            else:
                bias_weights = np.random.rand(self.out_features)
                self.verboseprint('*** bias ***')
                self.verboseprint('bias shape: {}'.format(bias_weights.shape))
        else:
            bias_weights = np.zeroes(self.out_features)
        self.verboseprint(bias_weights)
        self.verboseprint('\n')
                              
        ''' apply linear transformation '''
        output = _input @ weights.T + bias_weights
        self.verboseprint(output)
        self.verboseprint('\n')
        return output

### Standalone test (random input, weights, and bias)

In [30]:
in_samples = 128 # input samples
in_features = 20 # input features
out_features = 30 # output features
_input = np.random.rand(in_samples, in_features) # define a random input
_weights = np.random.rand(out_features, in_features) # define random weights
_bias = np.random.rand(out_features) # define random bias

In [36]:
# get Linear output with the random input

linear = Linear(in_features, out_features, verbose = True) # call an instance of the class with the input parameters
_output = linear.forward(_input, weights = _weights, bias_weights = _bias) # apply linear transformation
# _output = linear.forward(_input) # apply linear transformation
print("*** Linear output ***")
print(_output)

*** parameters ***
in_features: 20, out_features: 30, bias: True
device: None, dtype: None


*** weights ***
weights shape: (30, 20)
[[9.83022151e-01 3.92545891e-01 4.17296137e-01 5.51176419e-02
  1.50880478e-01 5.86725645e-01 8.30162111e-02 6.17526908e-02
  6.62458348e-01 9.45134295e-02 1.78971277e-01 9.20063607e-01
  9.15804574e-01 1.34192428e-02 3.78561532e-01 7.93727616e-01
  7.65419183e-01 4.23665830e-01 8.23077116e-01 7.64988014e-01]
 [3.78179003e-01 4.25781294e-01 4.13330975e-01 6.94117151e-01
  4.74332548e-01 2.69599871e-01 3.45855274e-01 9.32160703e-01
  9.28462513e-01 9.80391084e-02 2.43699174e-01 5.72328826e-01
  4.49787722e-01 1.50855086e-01 7.46840601e-01 5.93001793e-01
  8.33641289e-01 3.28482860e-01 8.69031175e-01 8.79293872e-01]
 [9.34603743e-01 7.81517049e-01 1.61816904e-01 9.44794821e-01
  2.27109998e-01 9.79636672e-01 7.21804516e-01 4.74233681e-01
  1.43161133e-01 2.32153747e-01 8.51276182e-01 8.26214651e-01
  5.98030885e-02 5.42760547e-01 5.66694263e-01 3.15704524e-

In [26]:
# get PyTorch output with the same random inputs as above

x = torch.DoubleTensor(_input)
A = torch.DoubleTensor(_weights)
b = torch.DoubleTensor(_bias)
output = torch.nn.functional.linear(x, A, bias = b)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[5.9197, 6.0041, 5.1066,  ..., 7.0375, 7.0805, 7.3120],
        [3.6620, 5.2423, 3.8472,  ..., 4.7562, 5.2316, 4.8721],
        [4.2813, 5.0258, 4.3232,  ..., 5.0951, 4.9565, 5.9333],
        ...,
        [4.8382, 5.4972, 4.4712,  ..., 5.7450, 4.3685, 6.0752],
        [5.0863, 4.9158, 3.6148,  ..., 5.6218, 5.3140, 5.4417],
        [4.9206, 5.2285, 4.6776,  ..., 6.1977, 5.6517, 6.8108]],
       dtype=torch.float64)


In [27]:
# compare outputs of Conv2D and PyTorch
print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

True


### Extensive tests (random input, weights, and bias)

In [38]:
def valid_params(num_tests):
    ''' generates `num_tests` number of valid input and kernel parameters '''
    
    params_list = []
    sample_count = 0
    while sample_count < num_tests:
        in_samples = np.random.randint(200)
        in_features = np.random.randint(100) + 1
        out_features = np.random.randint(100) + 1
        params_list.append({'in_samples': in_samples, 'in_features': in_features, 'out_features': out_features})
        sample_count += 1
    return params_list

In [42]:
def run_tests(num_tests):
    ''' sweep different input parameters and test by comparing outputs of ReLU and PyTorch '''
    
    num_passed = 0
    params_list = valid_params(num_tests)
    print('Number of tests: {}\n\n'.format(len(params_list)))

    for i, params in enumerate(tqdm(params_list)):
        print('Test: {}\nParams: {}'.format(i, params))
        in_samples = params['in_samples'] # input samples
        in_features = params['in_features'] # input features
        out_features = params['out_features'] # output features
        
        _input = np.random.rand(in_samples, in_features) # define a random input
        _weights = np.random.rand(out_features, in_features) # define random weights
        _bias = np.random.rand(out_features) # define random bias
        
        try:
            # get Linear output with the random input
            linear = Linear(in_features, out_features) # call an instance of the class with the input parameters
            _output = linear.forward(_input, weights = _weights, bias_weights = _bias) # apply linear transformation

            # get PyTorch output with the same random inputs as above
            x = torch.DoubleTensor(_input)
            A = torch.DoubleTensor(_weights)
            b = torch.DoubleTensor(_bias)
            output = torch.nn.functional.linear(x, A, bias = b)
            
        except Exception as e:
            print(e)
            print('Result: False\n\n') # treating exception as a failed test
            continue

        # compare outputs of Linear and PyTorch
        result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
        print('Result: {}\n\n'.format(result))
        if result:
            num_passed += 1

    print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests)))


In [43]:
num_tests = 1000
run_tests(num_tests)

Number of tests: 1000




  1%|▎                                         | 7/1000 [00:00<00:16, 58.77it/s]

Test: 0
Params: {'in_samples': 71, 'in_features': 40, 'out_features': 4}
Result: True


Test: 1
Params: {'in_samples': 0, 'in_features': 64, 'out_features': 69}
Result: True


Test: 2
Params: {'in_samples': 83, 'in_features': 43, 'out_features': 79}
Result: True


Test: 3
Params: {'in_samples': 151, 'in_features': 67, 'out_features': 78}
Result: True


Test: 4
Params: {'in_samples': 57, 'in_features': 53, 'out_features': 19}
Result: True


Test: 5
Params: {'in_samples': 151, 'in_features': 82, 'out_features': 67}
Result: True


Test: 6
Params: {'in_samples': 132, 'in_features': 86, 'out_features': 94}
Result: True


Test: 7
Params: {'in_samples': 112, 'in_features': 4, 'out_features': 59}
Result: True


Test: 8
Params: {'in_samples': 46, 'in_features': 100, 'out_features': 97}
Result: True


Test: 9
Params: {'in_samples': 57, 'in_features': 77, 'out_features': 11}
Result: True


Test: 10
Params: {'in_samples': 178, 'in_features': 42, 'out_features': 64}
Result: True


Test: 11
Params: 

  2%|▉                                        | 24/1000 [00:00<00:13, 70.00it/s]

Result: True


Test: 14
Params: {'in_samples': 1, 'in_features': 56, 'out_features': 11}
Result: True


Test: 15
Params: {'in_samples': 141, 'in_features': 27, 'out_features': 93}
Result: True


Test: 16
Params: {'in_samples': 43, 'in_features': 87, 'out_features': 76}
Result: True


Test: 17
Params: {'in_samples': 144, 'in_features': 92, 'out_features': 67}
Result: True


Test: 18
Params: {'in_samples': 127, 'in_features': 99, 'out_features': 11}
Result: True


Test: 19
Params: {'in_samples': 9, 'in_features': 86, 'out_features': 47}
Result: True


Test: 20
Params: {'in_samples': 12, 'in_features': 64, 'out_features': 58}
Result: True


Test: 21
Params: {'in_samples': 142, 'in_features': 61, 'out_features': 41}
Result: True


Test: 22
Params: {'in_samples': 153, 'in_features': 19, 'out_features': 60}
Result: True


Test: 23
Params: {'in_samples': 95, 'in_features': 74, 'out_features': 1}
Result: True


Test: 24
Params: {'in_samples': 166, 'in_features': 94, 'out_features': 88}
Result:

...

Result: True


Test: 928
Params: {'in_samples': 76, 'in_features': 6, 'out_features': 73}
Result: True


Test: 929
Params: {'in_samples': 55, 'in_features': 49, 'out_features': 14}
Result: True


Test: 930
Params: {'in_samples': 32, 'in_features': 100, 'out_features': 8}
Result: True


Test: 931
Params: {'in_samples': 195, 'in_features': 67, 'out_features': 60}
Result: True


Test: 932
Params: {'in_samples': 177, 'in_features': 58, 'out_features': 84}
Result: True


Test: 933
Params: {'in_samples': 14, 'in_features': 23, 'out_features': 49}
Result: True


Test: 934
Params: {'in_samples': 177, 'in_features': 71, 'out_features': 32}
Result: True


Test: 935
Params: {'in_samples': 114, 'in_features': 85, 'out_features': 33}
Result: True


Test: 936
Params: {'in_samples': 93, 'in_features': 15, 'out_features': 73}
Result: True


Test: 937
Params: {'in_samples': 65, 'in_features': 27, 'out_features': 71}
Result: True


Test: 938
Params: {'in_samples': 75, 'in_features': 27, 'out_features': 

 97%|█████████████████████████████████████▊ | 969/1000 [00:07<00:00, 153.99it/s]

Result: True


Test: 963
Params: {'in_samples': 7, 'in_features': 71, 'out_features': 22}
Result: True


Test: 964
Params: {'in_samples': 196, 'in_features': 94, 'out_features': 94}
Result: True


Test: 965
Params: {'in_samples': 58, 'in_features': 75, 'out_features': 78}
Result: True


Test: 966
Params: {'in_samples': 47, 'in_features': 79, 'out_features': 99}
Result: True


Test: 967
Params: {'in_samples': 44, 'in_features': 48, 'out_features': 75}
Result: True


Test: 968
Params: {'in_samples': 55, 'in_features': 79, 'out_features': 15}
Result: True


Test: 969
Params: {'in_samples': 40, 'in_features': 59, 'out_features': 48}
Result: True


Test: 970
Params: {'in_samples': 88, 'in_features': 67, 'out_features': 26}
Result: True


Test: 971
Params: {'in_samples': 189, 'in_features': 81, 'out_features': 89}
Result: True


Test: 972
Params: {'in_samples': 109, 'in_features': 78, 'out_features': 24}
Result: True


Test: 973
Params: {'in_samples': 115, 'in_features': 93, 'out_features': 

100%|██████████████████████████████████████| 1000/1000 [00:07<00:00, 133.58it/s]

Result: True


Test: 984
Params: {'in_samples': 60, 'in_features': 83, 'out_features': 61}
Result: True


Test: 985
Params: {'in_samples': 70, 'in_features': 47, 'out_features': 32}
Result: True


Test: 986
Params: {'in_samples': 138, 'in_features': 74, 'out_features': 62}
Result: True


Test: 987
Params: {'in_samples': 122, 'in_features': 52, 'out_features': 55}
Result: True


Test: 988
Params: {'in_samples': 163, 'in_features': 86, 'out_features': 98}
Result: True


Test: 989
Params: {'in_samples': 29, 'in_features': 87, 'out_features': 72}
Result: True


Test: 990
Params: {'in_samples': 97, 'in_features': 5, 'out_features': 69}
Result: True


Test: 991
Params: {'in_samples': 119, 'in_features': 8, 'out_features': 57}
Result: True


Test: 992
Params: {'in_samples': 47, 'in_features': 14, 'out_features': 15}
Result: True


Test: 993
Params: {'in_samples': 12, 'in_features': 4, 'out_features': 56}
Result: True


Test: 994
Params: {'in_samples': 179, 'in_features': 19, 'out_features': 1